### ML experimentation
Current approach:
- Use CodeBERT to generate embeddings
- Learn embeddings

**Problem**: Embeddings too big for memory.
**Possible solution**: Create embeddings on the fly during training...

_References_:
- https://www.analyticsvidhya.com/blog/2019/09/demystifying-bert-groundbreaking-nlp-framework/
- https://github.com/microsoft/CodeBERT

In [22]:
import pandas as pd
import numpy as np
import math
from transformers import AutoTokenizer, AutoModel
import torch

In [23]:
# Init CodeBERT
tokenizer_bert = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model_bert = AutoModel.from_pretrained("microsoft/codebert-base")

In [24]:
# DO NOT forget 'keep_default_na=False' --> otherwise some NaN values in read data
df = pd.read_csv('./big-vul_dataset/line_sample_10p_original_ratio.csv', skipinitialspace=True, low_memory=True, keep_default_na=False)

In [25]:
df.head()

,Unnamed: 0,Unnamed: 0.1,func_id,line,vul,idx_in_func
0,0,5816239,188076,{,1,687
1,1,5617986,184337,switch (point.m_state) {,1,5
2,2,5840278,188293,return (GetKind() == kBlockEOS);,1,2
3,3,5609610,183954,InputMethodDescriptor current_input_method_;,1,754
4,4,5642688,185008,"// No changes detected, tell the client that t...",1,22


In [27]:
from sklearn.model_selection import train_test_split

# split into training and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(df['line'], df['vul'], test_size=0.2, random_state=42)

print('X_tr shape:',X_tr.shape)

X_tr shape: (404186,)


In [37]:
def bert_encode(list_of_strings):
    embeddings = []
    for s in list_of_strings:
        # tokenize
        code_tokens=tokenizer_bert.tokenize(s)
        # add special tokens
        tokens=[tokenizer_bert.cls_token]+code_tokens+[tokenizer_bert.sep_token]
        # convert to IDs
        tokens_ids=tokenizer_bert.convert_tokens_to_ids(tokens)
        # create embedding
        context_embeddings=model_bert(torch.tensor(tokens_ids)[None,:])[0]
        embeddings.append(context_embeddings)
        
    return embeddings

In [38]:
X_tr_bert = bert_encode(X_tr.tolist())
X_val_bert = bert_encode(X_val.tolist())

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 245760 bytes.

In [ ]:
from sklearn.linear_model import LogisticRegression

# LR model
vul_model = LogisticRegression()
# train
vul_model = vul_model.fit(X_tr_bert, y_tr)
# predict
pred_vul = vul_model.predict(X_val_bert)

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, pred_bert))